In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")

Populating the interactive namespace from numpy and matplotlib


/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)


## Alignment matrices for final production

In [2]:
def create_clean_matrices(onco_in, ts_in, mut_out, aff_out):
    patient_mutations = pd.concat([pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/matrices/oncogenes.patient_mutation.csv', index_col=0).transpose(),
                                pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/matrices/tsgenes.patient_mutation.csv', index_col=0).transpose(),
                                pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/matrices/indels.patient_mutation.csv', index_col=0).transpose()]).transpose()
    patient_affinites = pd.concat([pd.read_csv(onco_in, index_col=0),
                                pd.read_csv(ts_in, index_col=0)]).transpose()
    print patient_affinites.shape, patient_mutations.shape
    # restrict to overlapping patients
    patients = [x for x in list(patient_affinites.index) if x in list(patient_mutations.index)]
    mutations = [x for x in list(patient_affinites.columns) if x in list(patient_mutations.columns)]
    patient_affinites = patient_affinites.ix[patients, mutations]
    patient_mutations = patient_mutations.ix[patients, mutations]
    print patient_affinites.shape, patient_mutations.shape
    # restrict to mutations >= 3 occurrences
    mutations = list(patient_mutations.sum()[patient_mutations.sum() > 2].index)
    print len(mutations)
    patient_affinites = patient_affinites.ix[:, mutations]
    patient_mutations = patient_mutations.ix[:, mutations]
    print patient_affinites.shape, patient_mutations.shape
    patient_affinites.to_csv(mut_out)
    patient_mutations.to_csv(aff_out)

In [5]:
# DR only
create_clean_matrices('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/oncogenes.DR.csv',
                      '/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/tsgenes.DR.csv',
                      '/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_affinities.cancer.DR.csv',
                      '/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_mutations.cancer.DR.csv')

(7842, 1245) (10177, 1585)
(7656, 1243) (7656, 1243)
762
(7656, 762) (7656, 762)


In [6]:
# all genes
create_clean_matrices('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/oncogenes.all.csv',
                      '/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/tsgenes.all.csv',
                      '/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_affinities.cancer.all.csv',
                      '/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_mutations.cancer.all.csv')

(7590, 1245) (10177, 1585)
(7409, 1243) (7409, 1243)
730
(7409, 730) (7409, 730)


Building

In [66]:
onco_in='/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/oncogenes.DR.csv'
ts_in='/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/tsgenes.DR.csv'
patient_mutations = pd.concat([pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/matrices/oncogenes.patient_mutation.csv', index_col=0).transpose(),
                            pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/matrices/tsgenes.patient_mutation.csv', index_col=0).transpose(),
                            pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/matrices/indels.patient_mutation.csv', index_col=0).transpose()]).transpose()
patient_affinites = pd.concat([pd.read_csv(onco_in, index_col=0),
                            pd.read_csv(ts_in, index_col=0)]).transpose()
print patient_affinites.shape, patient_mutations.shape
# restrict to overlapping patients
patients = [x for x in list(patient_affinites.index) if x in list(patient_mutations.index)]
mutations = [x for x in list(patient_affinites.columns) if x in list(patient_mutations.columns)]

(2098, 1245) (10177, 1585)


In [67]:
print len(patients), len(mutations)

2047 1243


Debugging

In [10]:
def PHBR(x):
    number_of_alleles = len(x)
    s = 0
    for i in range(number_of_alleles):
        s += 1/float(x[i])
    return number_of_alleles / float(s)

In [4]:
patient_dictionary = pickle.load(open('/cellar/users/ramarty/Data/hla_ii/hla_types/TCGA.HLA_classII.p'))

In [6]:
category = 'oncogenes'
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/allele_matrices/{0}.csv'.format(category), index_col=0)

In [11]:
patients_used = []
for patient in patient_dictionary.keys()[:50]:
    patient_alleles = patient_dictionary[patient][:2]
    try:
        df[patient] = df[patient_alleles].apply(PHBR, axis=1)
        patients_used.append(patient)
    except:
        print patient, patient_alleles

TCGA-FD-A43N ['DRB1_1596', 'DRB1_0701']


In [9]:
print len(patients_used)

0


In [12]:
alleles = [x.strip() for x in open('/cellar/users/ramarty/Data/hla_ii/presentation/other/netMHCIIpan_alleles.txt').readlines() if x.strip()[:2] != 'H-']
len(alleles)

5620

In [13]:
'DRB1_1596' in alleles

False

In [15]:
[x for x in alleles if 'DRB' in x]

['DRB1_0101',
 'DRB1_0102',
 'DRB1_0103',
 'DRB1_0104',
 'DRB1_0105',
 'DRB1_0106',
 'DRB1_0107',
 'DRB1_0108',
 'DRB1_0109',
 'DRB1_0110',
 'DRB1_0111',
 'DRB1_0112',
 'DRB1_0113',
 'DRB1_0114',
 'DRB1_0115',
 'DRB1_0116',
 'DRB1_0117',
 'DRB1_0118',
 'DRB1_0119',
 'DRB1_0120',
 'DRB1_0121',
 'DRB1_0122',
 'DRB1_0123',
 'DRB1_0124',
 'DRB1_0125',
 'DRB1_0126',
 'DRB1_0127',
 'DRB1_0128',
 'DRB1_0129',
 'DRB1_0130',
 'DRB1_0131',
 'DRB1_0132',
 'DRB1_0301',
 'DRB1_0302',
 'DRB1_0303',
 'DRB1_0304',
 'DRB1_0305',
 'DRB1_0306',
 'DRB1_0307',
 'DRB1_0308',
 'DRB1_0310',
 'DRB1_0311',
 'DRB1_0313',
 'DRB1_0314',
 'DRB1_0315',
 'DRB1_0317',
 'DRB1_0318',
 'DRB1_0319',
 'DRB1_0320',
 'DRB1_0321',
 'DRB1_0322',
 'DRB1_0323',
 'DRB1_0324',
 'DRB1_0325',
 'DRB1_0326',
 'DRB1_0327',
 'DRB1_0328',
 'DRB1_0329',
 'DRB1_0330',
 'DRB1_0331',
 'DRB1_0332',
 'DRB1_0333',
 'DRB1_0334',
 'DRB1_0335',
 'DRB1_0336',
 'DRB1_0337',
 'DRB1_0338',
 'DRB1_0339',
 'DRB1_0340',
 'DRB1_0341',
 'DRB1_0342',
 'DRB1